# Integrating your Devices with MADSci Nodes

This notebook aims to teach you how to automate and integrate all the devices, instruments, sensors, and robots in your self-driving lab using the MADSci Node standard.

## Goals

After completing this notebook, you should understand

1. What we mean when we talk about a MADSci Node
2. The MADSci Node interface standard
3. How to integrate and automate a device using the MADSci Node standard
4. How to use the `RestNode` python class included in `madsci.node_module` to integrate a MADSci Node


Outline:

- overview of madsci/nodes/terms
- dashboard x workcell demo
- code deep dive

# What is a Node?

- In MADSci, a `Node` refers to a single instrument, sensor, robot, or other device, combined with the software needed to control, operate, automate, and integrate it into the Automated or Autonomous Lab as a whole.
- Node's take **Action Requests**, and return **Action Results**. They also report **State** and **Status** information, and can optionally manage **Resources**.

<img src="./assets/mermaid/NodeIO.svg" width=1000 style="background-color: white"></img>

## Anatomy of a Node

A `Node` typically consists of the following sub-components:

- A physical **device** (robot, instrument, sensor, etc.)
- A **driver**, API, library, or software application for communicating with that device, often provided by the hardware vendor
- A device **interface** class that handles the neccessary initialization, communication, and cleanup required to use the given device
- The **node implementation**, typically a class, which uses the interface to handle the execution of the actions and the lifecycle of the node (state, status, statup, shutdown, etc.).
- The **node definition** and **configuration**, which define specific details about a given instance of a node and how it should. We typically implement these as .YAML files
- The **node server** and **node client**, which allow for standardized control of nodes. These are implementations of the MADSci Node standard interface, and operate using standard protocols (currently, REST-based HTTP)

<img src="./assets/mermaid/NodeDiagram.svg" width=1000></img>

In [ ]:
# Install dependencies
%pip install madsci.common madsci.node_module httpx

In [ ]:
from madsci.node_module.rest_node_module import RestNode
from madsci.common.types.node_types import NodeDefinition
from rich import print

In [ ]:
class ExampleRobotNode(RestNode):
    """Define an Example Robot Node. It doesn't do anything yet, but it's a good starting point."""

# In most cases, this node definition is a .yaml file that we pass as the "--definition" argument at runtime.
node_definition = NodeDefinition(
    node_name="example_node",
    module_name="example_node_module",
    description="An example node",
)
example_node = ExampleRobotNode(node_definition=node_definition)
# Normally, `start_node` starts a web server and listens for incoming requests.
# Here, we are just testing the node's functionality, so we set testing=True.
# This will run the node in a testing mode, which is useful for debugging.
example_node.start_node(testing=True)

In [ ]:
# Demo Magic to avoid having to actually run rest servers
import contextlib
from fastapi.testclient import TestClient

from collections.abc import Generator
from typing import Any
from unittest.mock import patch

from madsci.client.node.rest_node_client import RestNodeClient

@contextlib.contextmanager
def node_server(node: RestNode) -> Generator[TestClient, None, None]:
    """Mock server context manager."""

    test_client = TestClient(node.rest_api)

    with test_client as requests:
        # Mock the server's behavior
        yield requests

@contextlib.contextmanager
def node_client(node: RestNode, client: RestNodeClient) -> Generator[RestNodeClient, None, None]:
    """Mock client context manager."""

    with node_server(node) as requests, patch("madsci.client.node.rest_node_client.requests") as mock_requests:
            def post_no_timeout(*args: Any, **kwargs: Any) -> Any:
                kwargs.pop("timeout", None)
                return requests.post(*args, **kwargs)

            mock_requests.post.side_effect = post_no_timeout

            def get_no_timeout(*args: Any, **kwargs: Any) -> Any:
                kwargs.pop("timeout", None)
                return requests.get(*args, **kwargs)

            mock_requests.get.side_effect = get_no_timeout

            yield client


In [ ]:
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    # Example of a request to the node
    print(client.get_info())

In [ ]:
class ExampleNodeWithLifecycle(ExampleRobotNode):
    """Define an Example Node with a startup and shutdown handlers."""

    def startup_handler(self) -> None:
        """Handle the startup event."""
        self.logger.log_info("Node is starting up...")

    def shutdown_handler(self) -> None:
        """Handle the shutdown event."""
        self.logger.log_info("Node is shutting down...")

node_definition = NodeDefinition(
    node_name="example_node",
    module_name="example_node_module",
    description="An example node",
)
example_node = ExampleNodeWithLifecycle(node_definition=node_definition)
example_node.start_node(testing=True)

with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    # Example of a request to the node
    print(client.get_status())

In [ ]:
import time

class ExampleNodeWithUpdates(ExampleNodeWithLifecycle):
    """Define an Example Node that periodically updates it's status and public-facing state."""

    node_state = {
        "some_state_key": 0,
    }

    def state_handler(self) -> None:
        """This is where you can implement logic to periodically update the node's public-facing state information."""
        self.node_state = {"some_state_key": self.node_state["some_state_key"] + 1}

    def status_handler(self) -> None:
        """
        This is where you can implement logic to periodically update the node's status information.
        """
        if self.node_state["some_state_key"] % 2 == 0:
            self.node_status.busy = True # Illustrative purposes only
        else:
            self.node_status.busy = False


node_definition = NodeDefinition(
    node_name="example_node",
    module_name="example_node_module",
    description="An example node",
    config_defaults={
        "state_update_interval": 2, # Change how frequently, in seconds, the node state is updated
        "status_update_interval": 0.5, # Change how frequently, in seconds, the node status is updated
    }
)
example_node = ExampleNodeWithUpdates(node_definition=node_definition)
example_node.start_node(testing=True)

with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    # Example of a request to the node
    for i in range(10):
        print(client.get_state())
        print(client.get_status().busy)
        time.sleep(1)

In [ ]:
from madsci.node_module.helpers import action
from madsci.common.types.action_types import ActionSucceeded, ActionRequest, ActionFailed
from madsci.common.types.node_types import NodeStatus

class ExampleNodeWithAction(ExampleNodeWithLifecycle):
    """Define an example node with an action."""

    node_status = NodeStatus()

    @action(name="example_action")
    def example_action(self, arg1: str, arg2: int) -> str:
        """
        An example action that takes two parameters and returns a string.
        """
        self.logger.log(f"Action called with arg1: {arg1}, arg2: {arg2}")
        if arg2 < 0:
            return ActionFailed(errors=["arg2 must be non-negative"])
        return ActionSucceeded()


node_definition = NodeDefinition(
    node_name="example_node",
    module_name="example_node_module",
    description="An example node",
)
example_node = ExampleNodeWithAction(node_definition=node_definition)
example_node.start_node(testing=True)

with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    # Example of a request to the node
    time.sleep(1)  # Wait for the node to start
    request = ActionRequest(
        action_name="example_action",
        args={"arg1": "Hello", "arg2": 42},
    )
    # Send the action request to the node
    print(client.send_action(request))

# Lifecycle of an Action

![Action Flow Status](./assets/mermaid/ActionStatusFlow.svg)

In [ ]:
# What if we forget an argument?
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    # Example of a request to the node
    time.sleep(1)  # Wait for the node to start
    request = ActionRequest(
        action_name="example_action",
        args={"arg1": "Hello"},
    )
    # Send the action request to the node
    print(client.send_action(request))

In [ ]:
# What if the action failed?
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    time.sleep(1)  # Wait for the node to start
    client.get_status()
    request = ActionRequest(
        action_name="example_action",
        args={"arg1": "Hello", "arg2": -42},
    )
    # Send the action request to the node
    print(client.send_action(request))

In [ ]:
# What does the node info look like?
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    # Example of a request to the node
    print(client.get_info())

In [ ]:
# Admin Action Example: Locking and Unlocking the Node
with node_client(example_node, RestNodeClient(url="http://localhost:2000")) as client:
    # Example of a request to the node
    time.sleep(1)  # Wait for the node to start
    print(client.send_admin_command("lock"))
    request = ActionRequest(
        action_name="example_action",
        args={"arg1": "Hello", "arg2": 0},
    )
    # Send the action request to the node
    print(client.send_action(request))
    print(client.send_admin_command("unlock"))
    request = ActionRequest(
        action_name="example_action",
        args={"arg1": "Hello", "arg2": 0},
    )
    # Send the action request to the node
    print(client.send_action(request))